In [ ]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv

--2024-11-15 17:02:49--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/train_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144474 (141K) [text/plain]
Saving to: ‘train_split.csv’

train_split.csv     100%[===================>] 141.09K  --.-KB/s    in 0.004s  

2024-11-15 17:02:49 (30.7 MB/s) - ‘train_split.csv’ saved [144474/144474]

--2024-11-15 17:02:49--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/main/test_split.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35259 (34K) 

In [ ]:
import pandas as pd

# Load the training DataFrame
df = pd.read_csv('/content/train_split.csv')
print(df.head())

                                                text  Joy  Fear  Anger  \
0  The light would come all the way up to your ve...    0     1      0   
1                   Well, my birthday is in January.    0     0      0   
2  Tears in my eyes, too much on my mind, and dee...    0     1      0   
3  My eyes scanned quickly into every room, every...    0     1      1   
4  I felt the afterglow of the late afternoon sun...    1     0      0   

   Sadness  Surprise  
0        0         1  
1        0         0  
2        1         0  
3        0         0  
4        0         0  


In [ ]:
!pip install transformers
!pip install scikit-learn
!pip install datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import numpy as np
import torch
import requests
import zipfile
import pandas as pd
import shutil
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

In [ ]:

train_df = pd.read_csv('/content/train_split.csv')
test_df = pd.read_csv('/content/test_split.csv')

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        # Encode the text using the tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Preprocess the data
MAX_LEN = 128
train_texts = train_df['text'].values
train_labels = train_df[['Joy', 'Fear', 'Anger', 'Surprise', 'Sadness']].values
test_texts = test_df['text'].values
test_labels = test_df[['Joy', 'Fear', 'Anger', 'Surprise', 'Sadness']].values


train_dataset = EmotionDataset(train_texts, train_labels, tokenizer, MAX_LEN)
test_dataset = EmotionDataset(test_texts, test_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
# RoBERTa-based EmotionClassifier
class EmotionClassifier(torch.nn.Module):
    def __init__(self, model_name='roberta-base', num_labels=5):  # 5 labels for your task
        super(EmotionClassifier, self).__init__()
        self.roberta = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, input_ids, attention_mask):
        return self.roberta(input_ids, attention_mask).logits


model = EmotionClassifier(num_labels=5)
model = model.to(model.device)

optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.BCEWithLogitsLoss()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#Train the model
def train_epoch(model, data_loader, optimizer, criterion):
    model.train()
    running_loss = 0
    correct_preds = 0
    total_preds = 0

    for batch in data_loader:
        optimizer.zero_grad()

        # Move data to the device
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        # Forward pass
        outputs = model(input_ids, attention_mask)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate predictions
        preds = torch.sigmoid(outputs) > 0.5  # Convert logits to binary predictions
        correct_preds += torch.sum(preds == labels).item()
        total_preds += labels.numel()

    return running_loss / len(data_loader), correct_preds / total_preds

# Training loop
# for epoch in range(3):
#     loss, accuracy = train_epoch(model, train_loader, optimizer, criterion)
#     print(f'Epoch {epoch+1} - Loss: {loss:.4f}, Accuracy: {accuracy*100:.2f}%')

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
# drive.mount('/content/drive')

# # Define a path to save the model in Google Drive
# drive_save_directory = "/content/drive/My Drive/roberta_emotion_model"

# # Create the directory if it doesn't exist
# os.makedirs(drive_save_directory, exist_ok=True)


In [ ]:
# # Save the model and tokenizer to Google Drive
# model.roberta.save_pretrained(drive_save_directory)
# tokenizer.save_pretrained(drive_save_directory)

# print(f"Model and tokenizer saved to Google Drive at {drive_save_directory}")


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).numpy()  # Convert logits to binary predictions
    return {
        'macro_f1': f1_score(labels, preds, average='macro')
    }

In [ ]:
import os
import shutil
import zipfile
import requests
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

import os
os.environ["WANDB_DISABLED"] = "true"

# Step 1: Download the model from GitHub (corrected URL)
github_url = "https://github.com/AnandSwapnil/NLP_Assignment3/raw/main/model/roberta_emotion_model.zip"
model_zip_path = "/content/roberta_emotion_model.zip"
output_dir = "/content/roberta_emotion_model"

# Clean up if the directory already exists
if os.path.exists(output_dir):
    if os.path.isdir(output_dir):
        print(f"Clearing existing directory: {output_dir}")
        shutil.rmtree(output_dir)
    else:
        print(f"Removing existing file: {output_dir}")
        os.remove(output_dir)

print("Downloading the model from GitHub...")
response = requests.get(github_url)
with open(model_zip_path, "wb") as f:
    f.write(response.content)

print("Extracting the model...")
os.makedirs(output_dir, exist_ok=True)
try:
    with zipfile.ZipFile(model_zip_path, "r") as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"Model extracted to {output_dir}")
except zipfile.BadZipFile:
    print(f"Error: {model_zip_path} is not a valid zip file!")

# Step 2: Load the model from the extracted folder
print("Loading the model...")
model = AutoModelForSequenceClassification.from_pretrained(output_dir)

# Step 3: Define evaluation arguments
eval_training_args = TrainingArguments(
    output_dir='/content/eval_chkpts',  # Directory to save checkpoints (if needed)
    per_device_eval_batch_size=8,       # Evaluation batch size
    logging_dir='/content/eval_logs',   # Directory for logs
)

# Step 4: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=eval_training_args,
    eval_dataset=test_dataset,        # You need to define your test dataset here
    compute_metrics=compute_metrics    # Define your metric computation function
)

# Step 5: Evaluate the model on the test dataset
eval_results = trainer.evaluate()
print(f"Macro F1 Score on Test Data: {eval_results.get('eval_macro_f1', 'Metric not found')}")


Extracting the model...
Model extracted to /content/roberta_emotion_model
Loading the model...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Macro F1 Score on Test Data: 0.6533447878541186
